In [1]:
require('nn')
require('hdf5')
require('optim')

f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
n_reduced = 1001

X_valids = f:read("valid_in"):all()
Y_valids = f:read("valid_out"):all()


train_rnn_X = f:read('train_rnn_X'):all()
train_rnn_Y = f:read('train_rnn_Y'):all()
train_X_seqeuence =  f:read('train_X_sequence'):all()
train_Y  = f:read('train_Y'):all()
valid_reduced_X = f:read('valid_reduced_X'):all()
valid_reduced_Y = f:read('valid_reduced_Y'):all()
windows_train = f:read('windows_train'):all()
windows_valid = f:read('windows_valid'):all()
test = f:read('test'):all()
valid_kaggle_with_spaces = f:read('valid_kaggle_with_spaces'):all()
valid_kaggle_without_spaces = f:read('valid_kaggle_without_spaces'):all()
valid_answers = f:read('valid_answers'):all()
nfeatures = f:read('nfeatures'):all()
nclasses = f['nclasses']:all()




f:close()

window_size = X_train:size(2)
n_actual = nfeatures

...rs/austinshin/torch/install/share/lua/5.1/hdf5/group.lua:312: HDF5Group:read() - no such child 'train_input' for [HDF5Group 33554432 /]
stack traceback:
	[C]: in function 'error'
	...rs/austinshin/torch/install/share/lua/5.1/hdf5/group.lua:312: in function 'read'
	[string "require('nn')..."]:6: in main chunk
	[C]: in function 'xpcall'
	...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:209: in function <...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:173>
	...s/austinshin/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010e34dbc0: 

In [2]:
Y_train[1]


9987	


In [10]:

embedded_size = 100
D_h = 100

lookup1 = nn.LookupTable(nfeatures*window_size,embedded_size)
linear1 = nn.Linear(window_size*embedded_size, D_h)
linear2 = nn.Linear(D_h, n_actual)
h = nn.Sequential()
h:add(lookup1)
--lookup1.weight:renorm(2, 2, 1)
h:add(nn.Reshape(window_size*embedded_size, True))
h:add(linear1)
h:add(nn.Tanh())
h:add(linear2)
h:add(nn.LogSoftMax())
mse = nn.ClassNLLCriterion()
--mse = nn.CrossEntropyCriterion()
w, dl_dw = h:getParameters()

In [64]:
X_set = X_train
Y_set = Y_train

In [78]:
function feval(w_new)
   
    bsize= batch
    local idx = torch.randperm(X_set:size(1)):sub(1,bsize)

    local x = torch.Tensor(bsize, X_set:size(2))

    
    --y = y:reshape(Y_train:size(1), 1)
    
    local y_ = torch.Tensor(bsize, 1)

    for i=1,bsize do
        x[i] = X_set[idx[i]]
        y_[i] = Y_set[idx[i]]
    end
    
    y_ = y_:squeeze()
    
    local inputs = x
    local targets = y_
    -- reset gradients (gradients are always accumulated, to accommodate
    -- batch methods)
    dl_dw:zero()
    lookup1.weight:renorm(2, 1, 5)
    -- evaluate the loss function and its derivative with respect to x, given a mini batch
    local prediction = h:forward(inputs)
    local loss_w = mse:forward(prediction, targets)
    h:backward(inputs, mse:backward(prediction, targets))
    
    return loss_w, dl_dw
            
end

batch = 10000

-- cycle on data
for i = 1,10000 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,w)

    if i % 10 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] )
        -- print(sgd_params)
    end
end

loss for iteration 10 is 8.0765816430523	


loss for iteration 20 is 7.4150537908816	


loss for iteration 30 is 6.9328089127195	


loss for iteration 40 is 6.7381139648026	


loss for iteration 50 is 6.650017654967	


...

loss for iteration 9960 is 4.7997650062446	


loss for iteration 9970 is 4.7859553324679	


loss for iteration 9980 is 4.7535810518154	


loss for iteration 9990 is 4.7323634025245	


loss for iteration 10000 is 4.8328080211887	


In [42]:
function perplexity(input_set, output_set, kaggle, yes)
    sum = 0
    distributions = torch.Tensor(input_set:size(1), output_set:size(2))
    for i = 1,input_set:size(1) do
        a = (h:forward(input_set[i]))
        b = nn.Index(1):forward{a,output_set[i]}
        --print(b)
        distribution = (nn.LogSoftMax():forward(b))
        distributions[i] = torch.exp(distribution)
        if yes == 1 then
            answer = distribution[kaggle[i]]
            sum = sum + answer
        end
        
    end
    return torch.exp(-sum/input_set:size(1)), distributions
end


In [81]:
perp, dist = perplexity(X_valid, q_valid, kaggle_valid, 1)
print(perp)

4.9999308322859	


In [ ]:
torch.sum(dist[3])

In [ ]:
distributions:size()

In [27]:
function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [58]:
write2file(distributions, 'predictions.txt')